In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()
#import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()

  Activating environment at `~/Desktop/PHD2022/Autumn/CSE 579/CS_project/CS_project/Project.toml`


In [2]:
using LinearAlgebra
using Plots
#using ControlSystems

In [3]:
c = 0.0
m = 1.0
function point_mass(state, u)
    x = state[1]                #x_1 = ẋ
    ẋ = state[2]
    ẍ = -(c/m)*x[2] + u[1]/(m)  #x_2 = ẍ = (-c/m)ẋ + u/m 
    y = state[3]
    ẏ = state[4]
    ÿ = -(c/m)*x[4] + u[2]/(m)  #x_2 = ẍ = (-c/m)ẋ + u/m 
    # dx[3] = x[3]
    # dx[4] = -(c/m)*x[3] + u(t)/(m)  #x_2 = ẍ = (-c/m)ẋ + u/m
    return [ẋ; ẍ; ẏ; ÿ]
end

point_mass (generic function with 1 method)

#### Point Mass Discrete

In [4]:
n = 2 #n states
m = 1 #2 controls
dt = 0.01 #step size
H = 15.0  #Horizon
k_steps = trunc(Int, H/dt) 
#A = [0 1; 0 -c/m]
A = [0 0 1 0; 0 0 0 1; 0 0 (-c/m) 0; 0 0 0 (-c/m)]  #Input Jacobian
B1 = [(1/m) 0; 0 (1/m); 0 0; 0 0] #Control Jacobian for point mass 1
B2 = [0 0; 0 0; (1/m) 0; 0 (1/m)] #Control Jacobian for point mass 2
Ad = dt .* A + I    #discretize (zero order hold)
B1d = dt .*B1   #discrete (zero order hold)
B2d = dt .*B2   #discrete (zero order hold)
Q1 = [1 0 0 0; 0 1 0 0; 0 0 0 0; 0 0 0 0] #state cost for point mass 1
Q2 = [-1 0 0 0; 0 -1 0 0; 0 0 0 0; 0 0 0 0]   #state cost for point mass 2
R11 = [1 0; 0 1]    #Control cost for player 1
R22 = [1 0; 0 1]    #Contorl cost for player 2
R12 = [0 0; 0 0]    #Control cost for player 1 associated with player 2's controls
R21 = [0 0; 0 0];    #Control cost for player 2 associated with player 1's controls

In [5]:
x0 = [0, 1, 1, 0]
xref = [2; 1; 1; 2]
function cost(xref, x0)
    x_k = copy(xref - x0)
    c1 = 0
    c2 = 0
    for k in length(k_steps)
        u1 = -[0.1 0.1 0.1 0.0; 0.0 0.0 0.0 0.0]*x_k
        u2 = -[0.12 0.1 0.1 0.0; 0.0 0.0 0.0 0.0]*x_k
        x_k = Ad*x_k + B1d*u1 + B2d*u2
        c1 += transpose(x_k)*Q1*x_k + transpose(u1)*R11*u1 + transpose(u2)*R12*u2
        c2 += transpose(x_k)*Q2*x_k + transpose(u2)*R22*u2 + transpose(u2)*R21*u2
    end
    return c1,c2
end

cost (generic function with 1 method)

In [6]:
c1, c2 = cost(xref, x0)

(4.032404, -3.934804)

In [111]:
Bd

2-element Vector{Float64}:
 0.0
 0.01

In [234]:
# dt = 0.01
# A = [0 1; 0 -c/m]
# B = [0; 1/m]
# Ad = dt .* A + I #discrete
# Bd = dt .*B #discrete
function lqr!(T,n,m)
    Q = 20.0*I#[0.5 0; 0 0.3]
    R = 1.0*I#[0.5]
    V = copy(Q)
    #println(T[1])
    T[1] = T[1] #- 1
    K = zeros(T[1], n*m)

    for i in 1:T[1]
        K_i = -inv(R + transpose(Bd) * V * Bd) * transpose(Bd) * V * Ad
        #println("fff", K_i)
        V = Q + transpose(K_i) * R * K_i + transpose(Ad + Bd * K_i) * V * (Ad + Bd * K_i)
        K[i, :] = reshape(transpose(K_i), (n*m,1))
        #println("f",reshape(K[i, :], (2,4))
    end
    #println("lqr..")
    return K
end

lqr! (generic function with 2 methods)

In [235]:
tf = 15.0
t = Array(range(0, tf, step=0.01))
length(t)
T_all = [1000]
K = lqr!(T_all,n,m)
x0 = [0.; 0.; 0. ; 0.]
#x0 = [0.; 0.]
#xref = [5.; 0.]
xref = [5.; 0.; 5. ; 0.]
#Forward rollout starting at x0
xhist = zeros(T_all[1],n)
xhist[1,:] = x0
uhist = zeros(T_all[1]-1,m)
for t = 1:(T_all[1]-1)
    #uhist[t,:] = -K[t,:]*(xhist[t,:] - xref)
    uhist[t,:] .= reshape(K[end, :], (m, n))*(xhist[t,:] - xref)

    xhist[t+1,:] .= Ad*xhist[t,:] + reshape(Bd,(n,m))*uhist[t,:]
end
#K

In [236]:
xhist

1000×4 Matrix{Float64}:
 0.0         0.0          0.0         0.0
 0.00108836  0.217672     0.00131481  0.262961
 0.00429597  0.42385      0.00518979  0.512035
 0.00951044  0.619044     0.0114892   0.747841
 0.0166244   0.803745     0.0200832   0.970971
 0.0255352   0.978423     0.0308481   1.18199
 0.036145    1.14353      0.0436653   1.38145
 0.04836     1.29948      0.0584218   1.56985
 0.062091    1.44671      0.0750096   1.74771
 0.0772525   1.58559      0.0933256   1.91549
 0.093763    1.71651      0.113271    2.07365
 0.111545    1.83983      0.134753    2.22262
 0.130523    1.95588      0.15768     2.36282
 ⋮                                    
 4.99974     0.00026237   6.03999     0.000316959
 4.99975     0.000259688  6.03999     0.000313719
 4.99975     0.000257034  6.03999     0.000310512
 4.99975     0.000254406  6.03999     0.000307338
 4.99975     0.000251806  6.04        0.000304196
 4.99976     0.000249231  6.04        0.000301086
 4.99976     0.000246684  6.04        0

In [128]:
Q = 1.0*I#[0.5 0; 0 0.3]
R = 1.0*I#[0.5]
Kinf = dlqr(Ad,Bd,Q,R)

2×4 Matrix{Float64}:
 0.991377     1.72209      -3.55366e-13  6.203e-14
 3.16664e-12  8.60255e-13   0.991377     1.72209

In [129]:
K = lqr!(T_all)[end, :]

8-element Vector{Float64}:
 -0.9913771736849692
 -1.7220868293653293
  0.0
  0.0
  0.0
  0.0
 -0.9913771736849692
 -1.7220868293653293